In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files, drive
import pandas as pd
import re
from tqdm.auto import tqdm
import jsonlines
import nltk
nltk.download('punkt')


drive.mount('/content/drive')

Mounted at /content/drive


# getting some statistics from ChatGPT answers dataset

In [ ]:
with jsonlines.open('./drive/My Drive/chat_gpt_results_few_final.jsonlines', 'r') as jsonl_f:
  lst = [obj for obj in jsonl_f]

In [ ]:
from statistics import mean, median

mean_input = mean([len(l['input']) for l in lst])
mean_output = mean([mean([len(out) for out in l['outputs']]) for l in lst])
median_input = median([len(l['input']) for l in lst])
median_output = median([median([len(out) for out in l['outputs']]) for l in lst])

In [ ]:
print(mean_input, mean_output)
print(median_input, median_output)
print(len(lst))

2456.2667910447763 1057.0957711442786
2460.5 1051.5
536


# labeling references

In [ ]:
with jsonlines.open('./drive/My Drive/chat_gpt_results_few_final.jsonl', 'r') as jsonl_f:
  lst = [obj for obj in jsonl_f]

In [ ]:
new_jsonl_data = []

the following code allows you to label each element of the dataset manually

In [ ]:
i = 1 # index of the element of the dataset
inp = lst[i]['input']
obj1 = lst[i]['obj1']
obj2 = lst[i]['obj2']
output = lst[i]['outputs'][0]

cam_sents = inp.split('\n\n')[1].split('\n')
cs = {}
for s in cam_sents:
  cs[int(s.split('. ')[0])] = s.split('. ')[1]

mentioned_in_output = re.findall('\[[0-9]+\]', output)
mentioned_in_output = [int(item.strip('[').strip(']')) for item in mentioned_in_output]

sentences = dict(enumerate(nltk.sent_tokenize(output)))
length = [_ for _ in range(len(sentences))]

print('INPUT:') # prints only those phrases which are mentioned in the references
for k, v in cs.items():
  if k in mentioned_in_output:
    print(f'{k}: {v}')

print('\n\n\n')
print('OUTPUT:') # prints only those sentences of the output that contain references
for k, v in sentences.items():
  if '[' in v:
    print(f'{k}: {v}')



In [ ]:
new_output = {'text': output, 'sentences': {}}

for i in length:
  new_output['sentences'][i] = {'text': sentences[i], 'items': {}}

by default all references have a tag True

In [ ]:
new_output = {'text': output, 'sentences': {}}

for i in length:
  items = re.findall('\[[0-9]+\]', sentences[i])
  items = [int(i.strip('[').strip(']')) for i in items]
  if items:
    items = {i: True for i in items}
    new_output['sentences'][i] = {'text': sentences[i], 'items': items}
  else:
    new_output['sentences'][i] = {'text': sentences[i], 'items': {}}


you can change the tag by specifying the sentence index and the reference number which is in brackets as many time as it needs

In [ ]:
i_sent = 0 # sentence
i_item = 0 # reference
new_output['sentences'][i_sent]['items'][i_item] = False

In [ ]:
data2append = {'obj1': obj1,
               'obj2': obj2,
               'input': inp,
               'output':new_output}

In [ ]:
new_jsonl_data.append(data2append)

when you finished making changes run the following code

In [ ]:
with jsonlines.open('./drive/My Drive/chatgpt_results_checked_.jsonl', mode='w') as writer:
    writer.write_all(new_jsonl_data)

# evaluating precision of ChatGPT answers

In [ ]:
from collections import Counter as c

In [ ]:
with jsonlines.open('./drive/My Drive/chatgpt_results_checked.jsonl', 'r') as jsonl_f:
  lst = [obj for obj in jsonl_f]

In [ ]:
vals = []
for i in range(len(lst)):
  sents = lst[i]['output']['sentences']
  for ind in sents.keys():
    vals.extend(sents[ind]['items'].values())

In [ ]:
total = len(vals)
true = c(vals)[True]

print(true/total)

0.8979591836734694


# correcting output

In [ ]:
with jsonlines.open('./drive/My Drive/chatgpt_results_checked.jsonl', 'r') as jsonl_f:
  lst = [obj for obj in jsonl_f]

In [ ]:
new_outputs = []

In [ ]:
i = 0
print(lst[i]['input'])
print('\n\n')
output = lst[i]['output']
print(output['text'])
sents = output['sentences']
print('\n\n')

itms = []
ss = lst[i]['output']['sentences']
for ind in ss.keys():
  itms.extend(ss[ind]['items'].values())

if len(set(itms)) > 1: 
  for v in sents.values():
    print(v['text'])
    print(v['items'])
else: # if there is nothing to change (all tags are True) then it automatically appends in the list
  new_outputs.append(output['text'])
  print('done')

example:

In [ ]:
new_outputs.append('''Steel and timber are both materials commonly used in construction, but each has its own advantages and disadvantages.

Some argue that steel is lighter in weight than timber [1], performs better in earthquakes [2], and has greater durability [3]. It is also said to dissipate heat faster than high carbon steel [11] and is a cost-effective option for breakwalls [6]. However, others argue that steel structures can cause cracks from humidity [8] and may pose environmental concerns.

Timber, on the other hand, is said to be strong, light, and reliable [12][13], making it simpler and safer for on-site construction than steel or concrete [13]. It is also cheaper and easier to transport [15][19], and can be a more environmentally friendly alternative to concrete and steel [17]. However, timber may not be as durable as steel and can be susceptible to damage from pests or weather.

Overall, the choice between steel and timber depends on specific needs and preferences. For example, steel may be a better choice for break-walls, while timber may be preferred for trailer construction [15]. Examining the pros and cons of each material can help in making an informed decision.

Arguments used: 1, 2, 3, 6, 8, 11, 12, 13, 15, 17, 19.''')

when you finished making changes run the following code

In [ ]:
new_lst = []

for i in range(len(lst)):
  inp = lst[i]['input']
  obj1 = lst[i]['obj1']
  obj2 = lst[i]['obj2']
  output = new_outputs[i]
  new_lst.append({'obj1': obj1, 'obj2': obj2, 'input': inp, 'output': output})

with jsonlines.open('./drive/My Drive/work/chatgpt_results_correct.jsonl', mode='w') as writer:
    writer.write_all(new_lst)